<a href="https://colab.research.google.com/github/Pankotaiimre2001/xdlol/blob/master/PROJEKT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# === 0. KÖNYVTÁRAK IMPORTÁLÁSA ÉS BEÁLLÍTÁSOK ===
import requests                     # HTTP kérésekhez (weboldal letöltése)
from bs4 import BeautifulSoup       # HTML parse-oláshoz
import pandas as pd                 # Adatkezeléshez (DataFrame)
import re                           # Reguláris kifejezésekhez
import unicodedata                  # Unicode normalizáláshoz
from functools import lru_cache
!pip install huspacy
import huspacy
#huspacy.download()

import spacy                        # NLP feladatokhoz (lemmatizálás)
from sentence_transformers import SentenceTransformer # Beágyazások generálásához
import torch                        # PyTorch (gyakran a transformers könyvtárak függősége)

# Figyelmeztetések kikapcsolása (opcionális, ha zavaróak a könyvtárak figyelmeztetései)
import warnings
warnings.filterwarnings('ignore')

# Magyar spaCy modell betöltése
# Győződj meg róla, hogy telepítetted Colab-ban: !python -m spacy download hu_core_news_lg
try:
    nlp_hu = huspacy.load()
    print("Magyar spaCy modell (hu_core_news_lg) sikeresen betöltve.")
except OSError:
    print("HIBA: A magyar spaCy modell (hu_core_news_lg) nincs telepítve vagy nem tölthető be.")
    print("Google Colab-ban futtasd: !python -m spacy download hu_core_news_lg")
    print("Majd indítsd újra a futási környezetet (Runtime -> Restart session).")
    nlp_hu = None

# A cél URL
TARGET_URL = "https://index.hu/kulfold/2025/05/22/zsolyomi-zsolt-gyilkossag-homoszexualis-szereto-furdokad-biztonsagi-ov-miami-florida-eszbontok/"

@lru_cache(maxsize=1024)
def lemmatize_text_cached(text):
    return lemmatize_text_with_spacy(text)

# === 1. HTML TARTALOM LETÖLTÉSE ===
print(f"\n--- 1. HTML Tartalom Letöltése: {TARGET_URL} ---")
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
html_content_article = None
try:
    response = requests.get(TARGET_URL, headers=headers, timeout=15) # Timeout hozzáadása
    response.raise_for_status()
    html_content_article = response.text
    print(f"Az oldal sikeresen letöltve (hossz: {len(html_content_article)} karakter).")
except requests.exceptions.RequestException as e:
    print(f"Hiba történt az oldal letöltése közben: {e}")

# === 2. FŐ CIKKSZÖVEG KINYERÉSE A HTML-BŐL ===
# Ez a rész erősen függ az adott weboldal HTML szerkezetétől.
# Javasolt a böngésző Fejlesztői Eszközeivel (F12) megvizsgálni a céloldalt
# és azonosítani a fő cikkszöveget tartalmazó HTML elemeket (tagek, class-ok, id-k).
print("\n--- 2. Fő Cikkszöveg Kinyerése ---")
extracted_article_text_parts = []
if html_content_article:
    soup = BeautifulSoup(html_content_article, "lxml")

    # Specifikus próbálkozás a FrontiersIn oldalakhoz (ezeket finomítsd a böngészős vizsgálat alapján!)

    # Absztrakt
    abstract_section = soup.find('div', class_='JournalAbstract')
    if abstract_section:
        extracted_article_text_parts.append("Absztrakt: " + abstract_section.get_text(separator=" ", strip=True))

    # Kulcsszavak
    keywords_section = soup.find('div', class_='Keywords')
    if keywords_section:
        extracted_article_text_parts.append("Kulcsszavak: " + keywords_section.get_text(separator=" ", strip=True))

    # Fő cikktörzs (a `div[role="article"]` tűnik a legrelevánsabb konténernek)
    article_body = soup.find('div', class_='article-content') or soup.find('article')
    if article_body:
        paragraphs = article_body.find_all('p')
        print(f"Talált cikktartalom {len(paragraphs)} bekezdéssel.")
    else:
        print("Figyelem: Nem találtuk a szokásos cikk konténert. Próbálkozás az összes <p> taggel.")
        paragraphs = soup.find_all('p')

    relevant_paragraphs = [p.get_text(separator=" ", strip=True) for p in paragraphs]
    extracted_article_text_parts.extend(relevant_paragraphs)


    # Üres stringek eltávolítása és összefűzés
    full_extracted_text = " ".join(filter(None, extracted_article_text_parts))

    if not full_extracted_text.strip() and html_content_article:
        print("Figyelem: A célzott kinyerés nem adott eredményt, próbálkozás a teljes body szövegével.")
        full_extracted_text = soup.body.get_text(separator=" ", strip=True) if soup.body else "Nem sikerült szöveget kinyerni."
else:
    full_extracted_text = "Az oldal HTML tartalma nem érhető el."

print(f"Kinyert nyers szöveg hossza: {len(full_extracted_text)} karakter.")
print("Kinyert szöveg eleje (max 500 karakter):")
print(full_extracted_text[:500] + "..." if len(full_extracted_text) > 500 else full_extracted_text)

# === 3. SZÖVEG TISZTÍTÁSA ÉS ELŐFELDOLGOZÁSA ===
print("\n--- 3. Szöveg Tisztítása és Előfeldolgozása ---")

# Adat DataFrame-be helyezése
df_article_processed = pd.DataFrame({'id': [TARGET_URL], 'raw_extracted_text': [full_extracted_text]})

# --- Segédfüggvények a tisztításhoz ---
def clean_special_chars_and_normalize(text):
    if pd.isna(text):
        return ""
    text = str(text)
    text = unicodedata.normalize("NFKC", text) # Unicode normalizálás
    # HTML entitások dekódolása (ha a BeautifulSoup nem kezelte volna mindet)
    # from html import unescape
    # text = unescape(text) # Ezt óvatosan, lehet, hogy nem szükséges
    text = re.sub(r"\[\d+\]", "", text) # [1], [23] stílusú hivatkozások eltávolítása
    text = re.sub(r"\[\d+-\d+\]", "", text) # [1-5] stílusú hivatkozások eltávolítása
    text = re.sub(r"http\S+", "", text) # URL-ek eltávolítása
    # Csak betűk, számok, magyar ékezetes karakterek és alapvető írásjelek megtartása.
    # Ez a regex szigorúbb lehet, pl. a zárójeleket, idézőjeleket is megtarthatod, ha fontosak.
    text = re.sub(r"[^a-zA-Z0-9áéíóöőúüűÁÉÍÓÖŐÚÜŰ\s.,!?:;\-]", " ", text) # Figyelj a kötőjelre a végén
    text = " ".join(text.split()) # Extra szóközök eltávolítása
    return text.strip()

def lemmatize_text_with_spacy(text_to_lemmatize):
    if pd.isna(text_to_lemmatize) or not nlp_hu:
        return str(text_to_lemmatize) if not pd.isna(text_to_lemmatize) else ""

    doc = nlp_hu(str(text_to_lemmatize))
    # Csak bizonyos PoS tageket tartunk meg, pl. főnevek, melléknevek, igék (opcionális)
    # allowed_pos = ['NOUN', 'ADJ', 'VERB', 'ADV']
    # lemmas = [token.lemma_ for token in doc if token.pos_ in allowed_pos and not token.is_stop]
    lemmas = [token.lemma_ for token in doc if not token.is_punct and not token.is_space] # Lemmatizálunk, de a központozást és szóközöket kihagyjuk
    return " ".join(lemmas)

def split_text_to_chunks(text, max_chunk_tokens=500, model=None):
    """
    Szöveg darabolása kb. max_chunk_tokens hosszúságú darabokra.
    A darabolás mondatok szerint történik.
    """
    if not model:
        raise ValueError("A 'model' paraméter megadása kötelező, hogy tudjuk hány token egy-egy mondat.")

    sentences = text.split('.')  # Egyszerű mondatbontás
    chunks = []
    current_chunk = ""

    for sentence in sentences:
        candidate = (current_chunk + " " + sentence).strip()
        n_tokens = len(model.tokenize(candidate))

        if n_tokens <= max_chunk_tokens:
            current_chunk = candidate
        else:
            if current_chunk:
                chunks.append(current_chunk.strip())
            current_chunk = sentence.strip()

    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks


def get_average_embedding(text, model, max_chunk_tokens=500):
    chunks = split_text_to_chunks(text, max_chunk_tokens=max_chunk_tokens, model=model)
    print(f"{len(chunks)} darabra bontva a hosszú szöveg.")

    embeddings = model.encode(chunks)

    # Átlagolás (mean pooling)
    average_embedding = sum(torch.tensor(e) for e in embeddings) / len(embeddings)
    return average_embedding.numpy()

# --- Tisztítási lépések alkalmazása ---
df_article_processed['cleaned_text'] = df_article_processed['raw_extracted_text'].apply(clean_special_chars_and_normalize)
df_article_processed['normalized_text'] = df_article_processed['cleaned_text'].str.lower()

# Lemmatizálás (opcionális - kapcsold ki/be a következő sort, ha kell/nem kell)
# A SentenceTransformer modellek általában jobban működnek lemmatizálás nélkül.
# Ha használod, a 'final_processed_text' legyen a 'lemmatized_text'.
def lemmatize_texts_batch(texts, nlp_model):
    docs = nlp_model.pipe(texts, batch_size=16, disable=["ner"])  # Gyorsabb NER nélkül
    lemmatized_texts = []
    for doc in docs:
        lemmas = [token.lemma_ for token in doc if not token.is_punct and not token.is_space]
        lemmatized_texts.append(" ".join(lemmas))
    return lemmatized_texts

df_article_processed['lemmatized_text'] = lemmatize_texts_batch(df_article_processed['normalized_text'].tolist(), nlp_hu)
df_article_processed['final_processed_text'] = df_article_processed['lemmatized_text']

#df_article_processed['final_processed_text'] = df_article_processed['normalized_text'] # Lemmatizálás nélkül

print("Feldolgozott szöveg (első 500 karakter):")
final_text_sample = df_article_processed['final_processed_text'].iloc[0]
print(final_text_sample[:500] + "..." if len(final_text_sample) > 500 else final_text_sample)
# display(df_article_processed.head()) # DataFrame megjelenítése

# === 4. VEKTOR REPREZENTÁCIÓ (BEÁGYAZÁSOK) GENERÁLÁSA ===
print("\n--- 4. Vektor Reprezentáció Generálása ---")

# Beágyazó modell betöltése (ha még nem történt meg)
try:
    sbert_model # Ellenőrizzük, létezik-e már a változó
    print(f"A '{sbert_model_name}' beágyazó modell már be van töltve.")
except NameError:
    sbert_model_name = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
    # Alternatívák: 'sentence-transformers/distiluse-base-multilingual-cased-v1'
    print(f"Beágyazó modell betöltése: {sbert_model_name}")
    sbert_model = SentenceTransformer(sbert_model_name)

text_for_embedding = df_article_processed['final_processed_text'].iloc[0]
article_embedding_vector = None

if text_for_embedding and text_for_embedding.strip():
    print(f"Beágyazás generálása a feldolgozott szöveghez...")
    # A .encode() várhat stringet vagy stringek listáját.
    article_embedding_vector = get_average_embedding(text_for_embedding, sbert_model)

    print("Generált beágyazás alakja:", article_embedding_vector.shape)
    print("Beágyazás első 5 értéke:", article_embedding_vector[:5])

    # A beágyazást hozzáadhatod a DataFrame-hez vagy elmentheted
    # df_article_processed['embedding'] = [article_embedding_vector]
    # np.save('single_article_embedding.npy', article_embedding_vector)
else:
    print("Nincs feldolgozott szöveg a beágyazáshoz.")

print("\n--- Feldolgozás vége ---")

output_filename = "feldolgozott_szoveg.txt"

with open(output_filename, "w", encoding="utf-8") as f:
    f.write(df_article_processed['final_processed_text'].iloc[0])

print(f"A feldolgozott szöveg elmentve ide: {output_filename}")

output_filename = "nyers_szoveg.txt"

with open(output_filename, "w", encoding="utf-8") as f:
    f.write(df_article_processed['raw_extracted_text'].iloc[0])

print(f"A nyers szöveg elmentve ide: {output_filename}")


Magyar spaCy modell (hu_core_news_lg) sikeresen betöltve.

--- 1. HTML Tartalom Letöltése: https://index.hu/kulfold/2025/05/22/zsolyomi-zsolt-gyilkossag-homoszexualis-szereto-furdokad-biztonsagi-ov-miami-florida-eszbontok/ ---
Az oldal sikeresen letöltve (hossz: 101133 karakter).

--- 2. Fő Cikkszöveg Kinyerése ---
Figyelem: Nem találtuk a szokásos cikk konténert. Próbálkozás az összes <p> taggel.
Kinyert nyers szöveg hossza: 7065 karakter.
Kinyert szöveg eleje (max 500 karakter):
A következő oldal tartalma a kiskorúakra káros lehet. Ha korlátozná a korhatáros tartalmak elérését a gépén, használjon szűrőprogramot ! Az oldal tartalma az Mttv. által rögzített besorolás szerinti V. vagy VI. kategóriába tartozik. Kövesse az Indexet Facebookon is! Egy csendes Pest vármegyei faluból indult, és Miamiban kötött ki, ahol két idős férfi életét oltotta ki hidegvérrel. Zsólyomi Zsolt története a magyar kriminálisok sötét fejezetei közé tartozik. A gyilkos neve sokaknak lehet ismerős ...

--- 3. Sz